# 멋진 인공지능 작사가 만들기

## Step 1. 데이터 다운로드
https://www.kaggle.com/paultimothymooney/poetry/data  
사이트에서 Song Lyrics 데이터를 다운로드합시다.

## Step 2. 데이터 읽어오기

glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 합시다.

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='utf-8-sig') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['baby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine']


In [2]:
print("Examples:\n", raw_corpus[:10])

Examples:
 ['baby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine', 'Hangin pictures on my wall', 'Every Saturday Rap Attack Mr Magic Marley Marl', 'I let my tape rock til my tape popped', 'Smokin weed and Bambu sippin on Private Stock', 'Way back when I had the red and black lumberjack', 'With the hat to match', 'Remember Rappin Duke duhha duhha']


\ufeff문제 참조
https://redcarrot.tistory.com/216

In [3]:
print(txt_list[:1])

['/home/aiffel/aiffel/lyricist/data/lyrics/notorious_big.txt']


## Step 3. 데이터 정제

### 필요한 패키지 import 하기

In [4]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
# 대망의 텐서플로우!
import os

1 Physical GPUs, 1 Logical GPUs


preprocess_sentence()를 사용하여 지나치게 긴문장 정제하도록 하겠습니다.

In [36]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 해당 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'          #문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

토큰화 이후 토큰의 개수가 15개를 넘어가는 문장들을 제외하고 corpus에 저장하도록 하겠습니다.

In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) > 15: continue
        
    corpus.append(preprocess_sentence(sentence))

print("데이터 크기:", len(corpus))
print("Examples:")
corpus[:10]

데이터 크기: 168590
Examples:


['<start> baby it was all a dream <end>',
 '<start> i used to read word up magazine <end>',
 '<start> salt n pepa and heavy d up in the limousine <end>',
 '<start> hangin pictures on my wall <end>',
 '<start> every saturday rap attack mr magic marley marl <end>',
 '<start> i let my tape rock til my tape popped <end>',
 '<start> smokin weed and bambu sippin on private stock <end>',
 '<start> way back when i had the red and black lumberjack <end>',
 '<start> with the hat to match <end>',
 '<start> remember rappin duke duhha duhha <end>']

데이터가 잘 정제되었는지 다음을 통해 확인해보도록 하겠습니다.

In [38]:
long_corpus = []

for i in range(0,len(corpus)):
    if len(corpus[i].split())> 20:
        long_corpus.append(corpus[i])

long_corpus[:10]

['<start> hit me , baby , one more time oh , baby , baby , the reason i breathe is you <end>',
 '<start> why do tears come at night ? and they say she s so lucky , she s a star <end>',
 '<start> la , la , la , la la , la , la , la oh baby , baby , have you seen amy tonight ? <end>',
 '<start> la , la , la , la la , la , la , la so tell me if you see her <end>',
 '<start> oh baby , baby , baby . . . la , la , la , la la , la , la , la <end>',
 '<start> la , la , la , la la , la , la , la love me , hate me <end>',
 '<start> la , la , la , la la , la , la , la yeah love me , hate me <end>',
 '<start> we re just so pretty ! hey , don t you know that it s always the same ? <end>',
 '<start> we re just so pretty ! tell me , is it true that these men are from mars ? <end>',
 '<start> boys , then she s in control can t live with em , can t live without em pink <end>']

필터링을 하고 났는데도 여러가지 쓸모없는 반복이나 특수문자로 인하여 토큰의 갯수가 20이 넘는 코퍼스가 보이는 것같습니다!
다시 corpus_new에 토큰의 갯수가 15미만인 문장들만 담아주도록 하겠습니다.

In [8]:
corpus_new = []

for i in range(0,len(corpus)):
    if len(corpus[i].split()) > 15 : continue
    
    corpus_new.append(corpus[i])

In [9]:
corpus_new[:10]

['<start> baby it was all a dream <end>',
 '<start> i used to read word up magazine <end>',
 '<start> salt n pepa and heavy d up in the limousine <end>',
 '<start> hangin pictures on my wall <end>',
 '<start> every saturday rap attack mr magic marley marl <end>',
 '<start> i let my tape rock til my tape popped <end>',
 '<start> smokin weed and bambu sippin on private stock <end>',
 '<start> way back when i had the red and black lumberjack <end>',
 '<start> with the hat to match <end>',
 '<start> remember rappin duke duhha duhha <end>']

In [10]:
print(len(corpus_new))

156226


## Step 4. 평가 데이터셋 분리

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.

In [11]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_new)

[[   2   52   11 ...    0    0    0]
 [   2    4  285 ...    0    0    0]
 [   2 2881  481 ...    0    0    0]
 ...
 [   2    4   61 ...    0    0    0]
 [   2  112  632 ...    0    0    0]
 [   2    8   50 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f6f44699610>


토큰 확인하기

In [12]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [13]:
tensor

array([[   2,   52,   11, ...,    0,    0,    0],
       [   2,    4,  285, ...,    0,    0,    0],
       [   2, 2881,  481, ...,    0,    0,    0],
       ...,
       [   2,    4,   61, ...,    0,    0,    0],
       [   2,  112,  632, ...,    0,    0,    0],
       [   2,    8,   50, ...,    0,    0,    0]], dtype=int32)

소스 문장 및 타켓 문장 만들어주기 

In [14]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  52  11  53  25   9 360   3   0   0   0   0   0   0]
[ 52  11  53  25   9 360   3   0   0   0   0   0   0   0]


훈련 데이터와 평가 데이터 분리하기

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
enc_train, enc_val, dec_train, dec_val = train_test_split(
                                         src_input, tgt_input, test_size = 0.2)

In [17]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124980, 14)
Target Train: (124980, 14)


# Step 5. 인공지능 만들기

In [18]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [19]:
test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [20]:
print(BUFFER_SIZE)

124980


In [21]:
print(steps_per_epoch)

488


### 모델 생성하기

In [22]:
from tensorflow.keras.models import Model

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 750
hidden_size = 1500
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [24]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.1661741e-04, -1.4487586e-04,  2.3380650e-04, ...,
         -1.7036783e-04,  2.9531401e-04, -1.3240316e-04],
        [ 2.4894805e-04, -1.5867874e-04, -1.4352816e-05, ...,
         -2.3635796e-04,  4.1457886e-04, -2.2177149e-04],
        [ 3.5217236e-04, -5.3204270e-04, -1.1311997e-04, ...,
         -3.6291976e-04,  5.1643816e-04, -7.5931312e-05],
        ...,
        [ 1.4043397e-03, -1.8923432e-03,  3.2512592e-03, ...,
         -4.7428111e-04, -1.8803276e-03, -2.2153086e-03],
        [ 1.5105837e-03, -2.1046284e-03,  3.9165257e-03, ...,
         -6.2279444e-04, -2.4277342e-03, -2.9824059e-03],
        [ 1.5902165e-03, -2.3113268e-03,  4.5699361e-03, ...,
         -7.6696638e-04, -2.9819233e-03, -3.6766960e-03]],

       [[ 2.1661741e-04, -1.4487586e-04,  2.3380650e-04, ...,
         -1.7036783e-04,  2.9531401e-04, -1.3240316e-04],
        [ 7.6638360e-04, -4.2430643e-04,  5.8499508e-04, ...,
         -2.8842810e-04, 

In [25]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  9000750   
_________________________________________________________________
lstm (LSTM)                  multiple                  13506000  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  18006000  
_________________________________________________________________
dense (Dense)                multiple                  18013501  
Total params: 58,526,251
Trainable params: 58,526,251
Non-trainable params: 0
_________________________________________________________________


In [26]:
tf.test.is_gpu_available()
#GPU를 못잡길래 확인해보았습니다..!
#여러분들도 학습속도가 너무 느리다싶으면 확인해보세요!

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

### 모델 학습시키기

In [27]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss = loss, optimizer = optimizer)

In [28]:
histroy = model.fit(train_dataset, validation_data = test_dataset, epochs=10)

Epoch 1/10
488/488 [==============================] - 124s 255ms/step - loss: 3.3070 - val_loss: 2.9582
Epoch 2/10
488/488 [==============================] - 124s 254ms/step - loss: 2.8238 - val_loss: 2.7471
Epoch 3/10
488/488 [==============================] - 124s 255ms/step - loss: 2.5924 - val_loss: 2.6057
Epoch 4/10
488/488 [==============================] - 127s 260ms/step - loss: 2.3831 - val_loss: 2.4929
Epoch 5/10
488/488 [==============================] - 128s 262ms/step - loss: 2.1877 - val_loss: 2.4064
Epoch 6/10
488/488 [==============================] - 129s 264ms/step - loss: 2.0027 - val_loss: 2.3362
Epoch 7/10
488/488 [==============================] - 128s 262ms/step - loss: 1.8268 - val_loss: 2.2811
Epoch 8/10
488/488 [==============================] - 125s 256ms/step - loss: 1.6589 - val_loss: 2.2358
Epoch 9/10
488/488 [==============================] - 125s 255ms/step - loss: 1.5036 - val_loss: 2.2050
Epoch 10/10
488/488 [==============================] - 125s 255m

### 실제로 가사 생성해보기

In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [30]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> i wish ", max_len=20)

'<start> i wish that i could talk to you <end> '

# 결과 및 평가

LSTM을 사용하여 인공지능 작사가 모델을 만들어 보았습니다.
모델에서 여러가지 하이퍼파라미터를 바꾸어가며 학습을 시켰고,
(embedding_size = 750, hidden_size = 1500)을 적용했을때 좋은 결과를 얻을수 있었습니다.  
최종 결과에서 val_loss: 2.1881를 얻었습니다.  

또 만들어낸 모델로 문장을 생성하였을때,  
'i love' => 'i love you , i love you'  
'i wish' => 'i wish that i could talk to you'  
같은 매끄러운 문장들을 얻을 수 있었습니다.  